In [8]:
# install necessary packages
!pip install jupyter-dash
!pip install dash
!pip install dash_core_components
!pip install dash_html_components
!pip install dash_bootstrap_components
!pip install dash_table
!pip install plotly
!pip install googlemaps

You should consider upgrading via the '/Users/mackliniek/anaconda3/bin/python3 -m pip install --upgrade pip' command.


You should consider upgrading via the '/Users/mackliniek/anaconda3/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/mackliniek/anaconda3/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/mackliniek/anaconda3/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 189 kB 716 kB/s eta 0:00:01
You should consider upgrading via the '/Users/mackliniek/anaconda3/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/mackliniek/anaconda3/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/mackliniek/anaconda3/bin/python3 -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement webbrowser
ERROR: No matching distribution found for webbrowser
You should consider upgrading via the '/Users/mackliniek/anaconda3/bin/python3 -m pip install --u

In [28]:
# DASH STUFF
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import dash_table

# OTHER STUFF
# from program_functions import * # create python module with functions for functionlaity and import it over here
from datetime import date
from os import path
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import csv
from datetime import date, timedelta
from datetime import datetime
# import webbrowser module for opening pdf file
import webbrowser
# import pdf writer for writing death certificate
from print_cert import print_form as death_cert
# import location directions
from location_directions import *
# import modals class for popup window
from modals_class import *

# INITIALISATION VALUE FOR TAB
compare_value='tab-1'

# MAKE CLASS OBJECT FOR FINDING DIRECTIONS TO PATIENT WITH GOOGLE MAPS AND CLASS OBJECT FOR POPUP WINDOW
FLD = find_location_directions()
modal = Modals_Class().modal1


# MAKE INITIALISATION DATAFRAMES
df_patient_info = pd.DataFrame([("Name", "", "Location", ""),("Gender", "", "Address", ""),("Age", "", "Postal Code", ""),("Place of birth", "", "City", "")],index=[0,1,2,3], columns=[" ","Patient", "  ", "Location"])
df_intake = pd.DataFrame(index=[0], columns=["Reason of admission"])
df_restrictions = pd.DataFrame(index=[0], columns=["Restrictions"])
df_history = pd.DataFrame(index=[0,1,2,3,4,5], columns=["History"])
df_allergy = pd.DataFrame(index=[0], columns=["Allergy"])
df_medication = pd.DataFrame(index=[0,1,2,3], columns=["Medication"])

# MAKE LIST OF TIME FOR AUTOMATIC COMPLETION OF MEDICAL FORMS
list_time = [str(i)+":00" for i in range(0,24)]

#  GET USER NAME AND INFO.
if path.exists("user.txt"):
    with open ("user.txt", "r") as f:
        userdata = f.readlines()
else:
    print("userdata not specified, define userdata in text file user.txt")
name_MD = userdata[0].replace('\n', ' ')
specialisme = userdata[1]


# CREATE CLASS OBJECTS FOR EVERY MEDICAL INSTITU THAT YOU WORK AT
Inst1 = NH('www.medical-institute-1-website.com', 'username','password')
Inst2 = NH('www.medical-institute-2-website.com', 'username','password')
Inst3 = NH('www.medical-institute-3-website.com', 'username','password')
 
# DEFINE LIST OF HEALTH INSTITUTION OBJECTS
org_list = [None, Inst1, Inst2, Inst3]


# APP START UP
app = JupyterDash(__name__, suppress_callback_exceptions=True)


# APP LAYOUT
app.layout = html.Div(children=[
    # HEADING
    dbc.Row([
        dbc.Col(html.H1("Medical Chart Dashboard", style={'textAlign': 'center', 'font-family': 'sans-serif'}), width=12)
    ]),

    # FIRST ROW
    dbc.Row([
        html.Div(children=[
            # DROPDOWN MENU
            html.Div('Health institution'),
            html.Div(id='dd-output-container'),
            modal,
            # SEARCH PATIENT INPUT
            html.Div('Last name'),
            dcc.Input(id='input-name', type='text', autoComplete='off'),
            html.Div('Birth date'),
            dcc.DatePickerSingle(
                id='input-birthdate',
                min_date_allowed=date(1900, 8, 5),
                max_date_allowed=date(2002, 9, 19),
                display_format='D-M-Y',
                date=None
            ), 
            html.Br(), 
            dbc.Button('Search patient', id ='submit-button', n_clicks=0),
            html.Br(),
            ],
            style={'textAlign':'center', 'vertical-align': 'text-top','border-style': 'solid', 'display':'inline-block', "width": "20%"}),
        html.Div(children=[
            # OUTPUT OF PATIENT INFO
            dash_table.DataTable(
                id='table-patient_info',
                columns=[{"name": i, "id": i} for i in df_patient_info.columns],
                data=df_patient_info.to_dict('records'),
                # STYLING
                style_cell_conditional=[
                    {'if': {'column_id': ' '},
                     'width': '15%'},
                    {'if': {'column_id': 'Location'},
                     'width': '30%'},
                    {'if': {'column_id': '  '},
                     'width': '15%'},
                    {'if': {'column_id': 'Patient'},
                     'width': '30%'},
                ],
                style_header={
                'textAlign': 'center',
                'whiteSpace': 'normal',
                'height': 'auto',
            },
                style_data={
                'textAlign': 'left',
                'whiteSpace': 'normal',
                'height': 'auto',
            },
            )
            ],
            style={'textAlign':'center', 'vertical-align': 'text-top','border-style': 'solid', 'display':'inline-block', "width": "50%"}),
        html.Div(children=[
            html.Div('Directions'),
            html.Br(), 
            html.Div(id='Route'),
            html.Br(),
            html.Div('Time to get to medical institute'),
            html.Br(),
            html.Div(id='Duration'),
            html.Br(),
            html.Br(), 
                        ],
            style={'textAlign':'center', 'vertical-align': 'text-top','border-style': 'solid', 'display':'inline-block', "width": "10%"}),
    ]),
    html.Br(), 
    # SECOND ROW
    dbc.Row([
        html.Div(children=[
                # TAB FUNCTIONS
                dcc.Tabs(
                id="tabs-with-classes-f",
                value='tab2-1',
                parent_className='custom-tabs',
                className='custom-tabs-container',
                children=[
                    dcc.Tab(
                        label='Voice recognition',
                        value='tabf-1',
                        className='custom-tab',
                        selected_className='custom-tab--selected'
                    ),
                    dcc.Tab(
                        label='Financial registration',
                        value='tabf-2',
                        className='custom-tab',
                        selected_className='custom-tab--selected'
                    ),
                    dcc.Tab(
                        label='End of life care',
                        value='tabf-3', className='custom-tab',
                        selected_className='custom-tab--selected'
                    ),
                    dcc.Tab(
                        label='Treatment',
                        value='tabf-4',
                        className='custom-tab',
                        selected_className='custom-tab--selected'
                    ),
                ]),
            html.Div(id='tabs-content-classes-f'),
            html.Div(id='dd-output-container2'),
            html.Div(id='hidden-div1', style={'display':'none'}),
            html.Div(id='hidden-div2', style={'display':'none'}),
            html.Div(id='hidden-div3', style={'display':'none'}),
            html.Div(id='hidden-div4', style={'display':'none'}),
            html.Div(id='intermediate-value', style={'display': 'none'})
            ],
            # STYLING TAB FUNCTIONS
            style={'textAlign':'center', 'vertical-align': 'text-top', 'border-style': 'solid', 'display':'inline-block', "width": "80%"}),
    ]),
    html.Br(),
    #  THIRD ROW
    dbc.Row([
        html.Div(children=[
            # TABLES  REASON OF INTAKE
            dash_table.DataTable(
                id='table-intake',
                columns=[{"name": i, "id": i} for i in df_intake.columns],
                data=df_intake.to_dict('records'),
                # STYLING
                style_cell_conditional=[
                    {'if': {'column_id': 'Intake'},
                     'width': '10%'},
                ],
                style_header={
                'textAlign': 'center',
                'whiteSpace': 'normal',
                'height': 'auto',
            },
                style_data={
                'textAlign': 'left',
                'whiteSpace': 'normal',
                'height': 'auto',
            },
            ),
            # TABLES  RESTRICTIONS
            dash_table.DataTable(
                id='table-restrictions',
                columns=[{"name": i, "id": i} for i in df_restrictions.columns],
                data=df_restrictions.to_dict('records'),
                # STYLING
                style_cell_conditional=[
                    {'if': {'column_id': 'Restrictions'},
                     'width': '10%'},
                ],
                style_header={
                'textAlign': 'center',
                'whiteSpace': 'normal',
                'height': 'auto',
            },
                style_data={
                'textAlign': 'left',
                'whiteSpace': 'normal',
                'height': 'auto',
            },
            ),
            # TABLE HISTORY
            dash_table.DataTable(
                id='table-history',
                columns=[{"name": i, "id": i} for i in df_history.columns],
                data=df_history.to_dict('records'),
                # STYLING
                style_cell_conditional=[
                    {'if': {'column_id': 'History'},
                     'width': '10%'},
                ],
                style_header={
                'textAlign': 'center',
                'whiteSpace': 'normal',
                'height': 'auto',
            },
                style_data={
                'textAlign': 'left',
                'whiteSpace': 'normal',
                'height': 'auto',
            },
            ),
            # TABLES  ALLERGY
            dash_table.DataTable(
                id='table-allergy',
                columns=[{"name": i, "id": i} for i in df_allergy.columns],
                data=df_allergy.to_dict('records'),
                # STYLING
                style_cell_conditional=[
                    {'if': {'column_id': 'Allergy'},
                     'width': '10%'},
                ],
                style_header={
                'textAlign': 'center',
                'whiteSpace': 'normal',
                'height': 'auto',
            },
                style_data={
                'textAlign': 'left',
                'whiteSpace': 'normal',
                'height': 'auto',
            },
            ),
            ],
            style={'textAlign':'center', 'vertical-align': 'text-top','border-style': 'solid', 'display':'inline-block', "width": "20%"}),
        html.Div(children=[
            # TABS GRAPHS
            dcc.Tabs(
                id="tabs-with-classes",
                value='tab-1',
                parent_className='custom-tabs',
                className='custom-tabs-container',
                children=[
                    dcc.Tab(
                        label='Vital parameters',
                        value='tab-1',
                        className='custom-tab',
                        selected_className='custom-tab--selected'
                    ),
                    dcc.Tab(
                        label='Temp',
                        value='tab-4',
                        className='custom-tab',
                        selected_className='custom-tab--selected'
                    ),
                    dcc.Tab(
                        label='Weight',
                        value='tab-2',
                        className='custom-tab',
                        selected_className='custom-tab--selected'
                    ),
                    dcc.Tab(
                        label='Glucose',
                        value='tab-3', className='custom-tab',
                        selected_className='custom-tab--selected'
                    ),
                ]),
            html.Div(id='tabs-content-classes')
            ],
            # STYLING TAB GRAPHS
            style={'textAlign':'center', 'vertical-align': 'text-top', 'border-style': 'solid', 'display':'inline-block', "width": "60%"}),
    ])],
style={'textAlign': 'center'}
)

@app.callback(Output('tabs-content-classes-f', 'children'),
              Input('tabs-with-classes-f', 'value'))
def render_content(tab):
    if tab == 'tabf-1':
        return html.Div([
            html.H3('Voice recognition'),
            dbc.Button('Start voice recognition', id ='voice-button', n_clicks=0),
            html.Br(),
            html.Br(),
        ])
    elif tab == 'tabf-2':
        return html.Div([
            html.H3('Register an inquest'),
            dbc.Button('Inquest', id ='inquest-button', n_clicks=0),
            html.Br(),
            html.Br(),
            html.H3('Register other activities'),
            html.Br(),
            html.Br(),
            dbc.Button('Glucose', id ='glucose-button', n_clicks=0),
            html.Br(),
            html.Br(),
            dbc.Button('Medication error', id ='med-error-button', n_clicks=0),
            html.Br(),
            html.Br(),
        ])
    elif tab == 'tabf-3':
        return html.Div([
            html.H3('Select time of death'),
            dcc.Dropdown(id='dropdown-time', options=[{'label': i, 'value': i} for i in list_time]),
            html.Br(),
            html.Br(),
            dbc.Button('Create death certificate for check', id ='certificate-button', n_clicks=0),
            html.Br(),
            html.Br(),
            dbc.Button('Create printable pdf', id ='certificate-print-pdf-button', n_clicks=0),
            html.Br(),
        ])
    elif tab == 'tabf-4':
        return html.Div([
            html.H3('Under construction')
        ])

@app.callback(
    Output("modal", "is_open"), 
    Output(component_id='dd-output-container', component_property = 'children'),
    [Input("open", "n_clicks"), Input("close", "n_clicks"), Input(component_id='NH', component_property ='value')],
    [State("modal", "is_open")],
    )
def toggle_modal(n1, n2, org_choice, is_open):
    org_choice_old = None
    if org_choice != org_choice_old:
        org_choice_old = org_choice
        org_choice = int(org_choice)
        # Actual login command
        # login into medical instute
        for i,j in enumerate(org_list):
            if org_choice == i:
                org_list2 = [" ", "Inst1", "Inst2", "Inst3"]
                string = org_list2[i]
    else:
        string = " "
    if n1 or n2:
        return not is_open, string
    return is_open, string

# TAB END OF LIFE
@app.callback(
    Output(component_id='dd-output-container2', component_property = 'children'),
    Input('certificate-button', 'n_clicks'),
    Input('certificate-print-pdf-button', 'n_clicks'),
    State('dropdown-time', 'value'),
    State('NH', 'value'),
    )
def end_life_care(n_clicks1, n_clicks2, time_value, org_choice):
    if n_clicks1>0:
        global name, gender, age, birthdate, MD, location, speciality, address, postal_code, place, first_name, birth_date_place, country
        DC = death_cert(name, gender, birthdate, MD, location, speciality, address, postal_code, place, first_name, birth_date_place, country, time_value)
        DC.main()
        webbrowser.open(r'Cert_for_check.pdf')
    if n_clicks2>0:        
        DC.final_pdf()
        webbrowser.open(r'Cert_for_print.pdf')
        
# TAB FINANCE
@app.callback(
    Output('hidden-div2', 'children'),
    Input('inquest-button', 'n_clicks'),
    Input('glucose-button', 'n_clicks'),
    Input('med-error-button', 'n_clicks'),
    State('NH', 'value'),
    )
def finance(n_clicks1, n_clicks2, n_clicks3, org_choice):
    if n_clicks1>0:
        # do finance
        pass
    if n_clicks2>0:        
        # do finance
        pass
    if n_clicks3>0:
        # do finance
        pass

        
@app.callback(
    Output(component_id='table-patient_info', component_property = 'data'),
    Output(component_id='table-patient_info', component_property = 'columns'),
    Output(component_id='table-intake', component_property = 'data'),
    Output(component_id='table-intake', component_property = 'columns'),
    Output(component_id='table-restrictions', component_property = 'data'),
    Output(component_id='table-restrictions', component_property = 'columns'),
    Output(component_id='table-history', component_property = 'data'),
    Output(component_id='table-history', component_property = 'columns'),
    Output(component_id='table-allergy', component_property = 'data'),
    Output(component_id='table-allergy', component_property = 'columns'),
    Output('intermediate-value', 'children'),
    Output('Route', 'children'), # gezamenlijke output
    Output('Duration', 'children'), # gezamenlijke output
    Input('submit-button', 'n_clicks'), # input search click
    State('NH', 'value'),
    State('input-name', 'value'),
    State('input-birthdate', 'date'),
    )
def search_pt(n_clicks, org_choice, name_passing_var, birthdate_passing_var):
    global compare_value, name, gender, age, birthdate, MD, location, speciality, address, postal_code, place, first_name, birth_date_place, country
    if (n_clicks > 0):
        if birthdate_passing_var and org_choice is not None:
            print("route 1")
            # make data global so its usable in other functions
            global data1, columns1, data2, columns2, data3, columns3, data4, columns4, data5, columns5, fig_graph_vit_par, fig_graph_weight, fig_graph_gluc

            # SEARCH PATIENT
            org_list[org_choice].search_pt(org_choice, birthdate_passing_var)
            # CHOOSE PATIENT IN EHR
            org_list[org_choice].choose_pt(org_choice, name_passing_var)
            # COPY NON-MEDICAL PATIENT  INFO
            name, gender_browser, age, birthdate, MD, location, speciality, address, postal_code, place, first_name, birth_date_place, country = org_list[org_choice].get_non_med_info(org_choice)
            if gender_browser == int(1):
                gender = "Female"
            else:
                gender = "Male"
            datetime_object = datetime.strptime(birthdate, '%d-%m-%Y')
            age = (date.today() - datetime.date(datetime_object)) // timedelta(days=365.2425)
            list_variables = [("Name", [name, (", "),first_name], "Nursery Home", location),("Gender", gender, "Address", address),("Age", [(str(age)+" years"), ("\n"+birthdate)], "Postal Code", postal_code),("Place of birth", birth_date_place, "City", place)]
            df_patient_info = pd.DataFrame(list_variables, columns=[" ","Patient", "  ", "Location"])
            data1 = df_patient_info.to_dict('records')
            columns1 =[{"name": i, "id": i} for i in df_patient_info.columns]
            
            # CREATE LINK FOR DIRECTIONS AND CALCULATE TRAVEL TIME
            duration, link_directions = FLD.create_directions(address, postal_code, place)

            # COPY MEDICAL PATIENT INFO
            df_intake, df_restrictions, df_history, df_allergy, df_vital_par, df_weight, df_glucose = org_list[org_choice].get_med_info(org_choice)

            # OPEN CSV FILE AND GET REASON OF INTAKE
            data2 = df_intake.to_dict('records')
            columns2 =[{"name": i, "id": i} for i in df_intake.columns]

            # OPEN CSV FILE AND GET RESTRICTIONS
            df_restrictions = df_restrictions.rename(columns={"text": "Restrictions"})
            data3 = df_restrictions.to_dict('records')
            columns3 =[{"name": i, "id": i} for i in df_restrictions.columns]        

            # OPEN CSV FILE AND IMPORT MEDICAL HISTORY  
            data4 = df_history.to_dict('records')
            columns4 =[{"name": i, "id": i} for i in df_history.columns]

            # OPEN CSV FILE AND GET ALLERGY        
            data5 = df_allergy.to_dict('records')
            columns5 =[{"name": i, "id": i} for i in df_allergy.columns]

            list_graphs_content = [df_vital_par, df_weight, df_glucose ]
            print("route 1a")
            list_graphs_content_json = []
            for item in list_graphs_content:
                new_item = item.to_json(date_format='iso', orient='split')
                list_graphs_content_json.append(new_item)
            
            return data1, columns1, data2, columns2, data3, columns3, data4, columns4, data5, columns5, list_graphs_content_json, html.A('click for route', href = link_directions, target="_blank"), html.Div(duration),

        # INITIALISATION
    if n_clicks == 0:
            print("route 0")
            # CREATE EMPTY PANDAS DFs
            df_patient_info = pd.DataFrame([("Name", "", "Nursery Home", ""),("Gender", "", "Address", ""),("Age", "", "Postal Code", ""),("Place of birth", "", "City", "")],index=[0,1,2,3], columns=[" ","Patient", "  ", "Location"])
            df_intake = pd.DataFrame(index=[0], columns=["Reason of admission"])
            df_restrictions = pd.DataFrame(index=[0], columns=["Restrictions"])
            df_history = pd.DataFrame(index=[0,1,2,3,4,5], columns=["History"])
            df_allergy = pd.DataFrame(index=[0], columns=["Allergy"])
            df_medication = pd.DataFrame(index=[0,1,2,3], columns=["Medication"])
            
            # CREATE EMPTY FIGURE
            
            data1 = df_patient_info.to_dict('records')
            columns1 =[{"name": i, "id": i} for i in df_patient_info.columns]

            data2 = df_intake.to_dict('records')
            columns2 =[{"name": i, "id": i} for i in df_intake.columns]

            data3 = df_restrictions.to_dict('records')
            columns3 =[{"name": i, "id": i} for i in df_restrictions.columns]

            data4 = df_history.to_dict('records')
            columns4=[{"name": i, "id": i} for i in df_history.columns]

            data5 = df_allergy.to_dict('records')
            columns5 =[{"name": i, "id": i} for i in df_allergy.columns]
            
            list_tabs = ['tab-1', 'tab-2', 'tab-3', 'tab-4']
            list_graphs_id = ['vit-par', 'weight', 'glucose', 'other']
            
            list_graphs_content = None
            
            print("route 0a")
            
            return data1, columns1, data2, columns2, data3, columns3, data4, columns4, data5, columns5, list_graphs_content, html.Div(" "), html.Div(" "),
    
@app.callback(
    Output('tabs-content-classes', 'children'), # gezamenlijke output 
    Input('tabs-with-classes', 'value'), # input tab choice click
    Input('intermediate-value', 'children')
)    
def graph_tabs(tab, list_graphs_content):
    list_tabs = ['tab-1', 'tab-2', 'tab-3', 'tab-4']
    list_graphs_id = ['vit-par', 'weight', 'glucose', 'other']
    global compare_value
    # INITIALISATION
    print("route 0c")
    if tab == compare_value and list_graphs_content == None:
            print("route 0d", list_graphs_content)
            # CREATE EMPTY PANDAS DFs
            df_patient_info = pd.DataFrame([("Name", "", "Medical Institute", ""),("Gender", "", "Address", ""),("Age", "", "Postal Code", ""),("Place of birth", "", "City", "")],index=[0,1,2,3], columns=[" ","Patient", "  ", "Location"])
            df_intake = pd.DataFrame(index=[0], columns=["Reason of admission"])
            df_restrictions = pd.DataFrame(index=[0], columns=["Restrictions"])
            df_history = pd.DataFrame(index=[0,1,2,3,4,5], columns=["History"])
            df_allergy = pd.DataFrame(index=[0], columns=["Allergy"])
            df_medication = pd.DataFrame(index=[0,1,2,3], columns=["Medication"])

            # CREATE EMPTY FIGURE
            fig_graph = go.Figure()
            
            data1 = df_patient_info.to_dict('records')
            columns1 =[{"name": i, "id": i} for i in df_patient_info.columns]

            data2 = df_intake.to_dict('records')
            columns2 =[{"name": i, "id": i} for i in df_intake.columns]

            data3 = df_restrictions.to_dict('records')
            columns3 =[{"name": i, "id": i} for i in df_restrictions.columns]

            data4 = df_history.to_dict('records')
            columns4=[{"name": i, "id": i} for i in df_history.columns]

            data5 = df_allergy.to_dict('records')
            columns5 =[{"name": i, "id": i} for i in df_allergy.columns]
            for i in range(0,4):
                if tab == list_tabs[i]:
                    return html.Div([
                    dcc.Graph(
                        id=list_graphs_id[i],
                        figure=fig_graph),
                    ]),

    # AFTER PATIENT SEARCH
    if list_graphs_content != None:
            print("route 1b")
                        # CREATE GRAPH CHART
            list_graphs_content_new = []
            for item in list_graphs_content:
                new_item = pd.read_json(item, orient='split')
                list_graphs_content_new.append(new_item)
            df_vital_par, df_weight, df_glucose = list_graphs_content_new
            fig_graph_vit_par = go.Figure()
            fig_graph_vit_par.add_trace(go.Scatter(x=df_vital_par["Date"], y=df_vital_par["Saturatie (%)"],
                                        mode='lines+markers',
                                        name='Saturation (%)',
                                        line=dict(color='deepskyblue', width=2),
                                        connectgaps=True,))
            fig_graph_vit_par.add_trace(go.Scatter(x=df_vital_par["Date"], y=df_vital_par["Pols (/min)"],
                                        mode='lines+markers',
                                        name='Pulse (bpm)',
                                        line=dict(color='red', width=2),
                                        connectgaps=True,))
            fig_graph_vit_par.add_trace(go.Scatter(x=df_vital_par["Date"], y=df_vital_par["Diastolic BP"],
                                        mode='lines+markers',
                                        name='Diastolic BP (mmHg)',
                                        line=dict(color='darkgreen', width=2),
                                        connectgaps=True,))
            fig_graph_vit_par.add_trace(go.Scatter(x=df_vital_par["Date"], y=df_vital_par["Systolic BP"],
                                        mode='lines+markers',
                                        name='Systolic BP (mmHg)',
                                        line=dict(color='forestgreen', width=2),
                                        connectgaps=True,))
            fig_graph_vit_par.add_trace(go.Scatter(x=df_vital_par["Date"], y=df_vital_par["Temp. (ᵒC)"],
                                        mode='lines+markers',
                                        name='Temp. (ᵒC)',
                                        line=dict(color='darkorange', width=2),
                                        connectgaps=True,))
            fig_graph_vit_par.add_trace(go.Scatter(x=df_vital_par["Date"], y=df_vital_par["AF (/min)"],
                                        mode='lines+markers',
                                        name='AF (/min)',
                                        line=dict(color='steelblue', width=2),
                                        connectgaps=True,))
            
            fig_graph_temp = go.Figure()            
            fig_graph_temp.add_trace(go.Scatter(x=df_vital_par["Date"], y=df_vital_par["Temp. (ᵒC)"],
                                        mode='lines+markers',
                                        name='Temp. (ᵒC)',
                                        line=dict(color='darkorange', width=2),
                                        connectgaps=True,))

            fig_graph_weight = go.Figure()
            fig_graph_weight.add_trace(go.Scatter(x=df_weight["Date"], y=df_weight["Gewicht (kg)"],
                                        mode='lines+markers',
                                        name='Weight kg',
                                        line=dict(color='deepskyblue', width=2),
                                        connectgaps=True,))
            
            fig_graph_gluc = go.Figure()
            fig_graph_gluc.add_trace(go.Scatter(x=df_glucose["Date"], y=df_glucose["column1"],
                                        mode='markers',
                                        name='First measurement glucose',
                                        line=dict(color='black', width=2),
                                        connectgaps=False,))     
            for i in range(2,5):
                try: 
                    fig_graph_gluc.add_trace(go.Scatter(x=df_glucose["Date"], y=df_glucose["column"+str(i)],
                                                mode='markers',
                                                name='Other day measurements',
                                                line=dict(color='blue', width=2),
                                                connectgaps=False,))    
                except:
                    pass

            fig_graph = go.Figure()
            list_tabs = ['tab-1', 'tab-2', 'tab-3', 'tab-4']
            list_graphs_id = ['vit-par', 'weight', 'glucose', 'temp']
            list_graphs_content = [fig_graph_vit_par, fig_graph_weight, fig_graph_gluc, fig_graph_temp]
            fig_graph = go.Figure()        
            for i in range(0,4):
                if tab == list_tabs[i]:
                    return html.Div([
                                dcc.Graph(
                                    id=list_graphs_id[i],
                                    figure=list_graphs_content[i]),
                                ]),
    
    # TAB CHANGE
    if not tab == compare_value:
            print("route1c")
            compare_value = tab
            fig_graph = go.Figure()
            try:
                for i in range(0,4):
                    if tab == list_tabs[i]:
                        return html.Div([
                        dcc.Graph(
                            id=list_graphs_id[i],
                            figure=list_graphs_content[i]),
                        ]) 
            except:
                for i in range(0,4):
                    if tab == list_tabs[i]:
                        return html.Div([
                        dcc.Graph(
                            id=list_graphs_id[i],
                            figure=fig_graph),
                        ])



if __name__ == '__main__':
    app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/
route 0
route 0a
route 0c
route 0d None
